In [67]:
import json
import pandas as pd
import requests

from domain.entities import *
from repository.sqlalchemy import *
from utility.changecapture import *
from utility.filehandler import *

CC_SOURCE = 'sensorar-application-1'
CC_KEYS = ['result']

TTN_CLUSTER_REGION = 'NAM1'
TTN_APP_NAME = 'sensorar-application-1'
TTN_API_KEY = 'NNSXS.2X5P5567GPCXENQYCPINZ5BAPA62GEC73R5NSVY.DJ7PAPBZDG6WV3FDSDMMRD4L3LPSDQBJNFK2BLLXTF6PXZXHKJVA'


def _request_uplink_messages_response():
    
    headers = {
        'Authorization': f'Bearer { TTN_API_KEY }',
        'Accept': 'text/event-stream'
    }

    params ={
        'after': '2020-08-20T00:00:00Z',
        'field_mask': 'up.uplink_message.decoded_payload'
    }
   
    response = requests.get(
        (
            f'https://{ str(TTN_CLUSTER_REGION).lower() }.cloud.thethings.network/api/v3/as'
            f'/applications/{ TTN_APP_NAME }/packages/storage/uplink_message'
        ),
        headers=headers,
        params=params
    )
    
    return response

def _assemply_response_uplink_messages(uplink_messages_response):
    
    content_str = '[' + uplink_messages_response.content.decode("utf-8") + ']'
    content_str = content_str.replace('\n', '')
    content_str = content_str.replace('{"result":', ',{"result":')
    content_str = content_str.replace('[,{"result":', '[{"result":')

    response_uplink_messages = json.loads(content_str)
    return response_uplink_messages    


def _cache_curr_uplink_messages(response_uplink_messages):
    FileHandler.save_json_file(response_uplink_messages, '../data/curr_upstream_records.json')

def _load_cached_uplink_messages_df():
    cached_uplink_messages_df = pd.read_json('./data/last_upstream_records.json', orient='records')
    return cached_uplink_messages_df


uplink_messages_response = _request_uplink_messages_response()
response_uplink_messages = _assemply_response_uplink_messages(uplink_messages_response)
_cache_curr_uplink_messages(response_uplink_messages)

curr_df = pd.read_json('../data/curr_upstream_records.json', orient='records')
curr_df = curr_df.astype(str)  
     
last_df = pd.read_json('../data/last_upstream_records.json', orient='records')
last_df = last_df.astype(str)  

inserts_df = ChangeCapture.get_wide_inserts_df(curr_df.copy(), last_df.copy(), CC_KEYS, CC_SOURCE)

samples = []
for i in range(len(inserts_df)):

    temp_insert_dic = json.loads(str(inserts_df.loc[i, 'result']).replace('\'', '\"'))

    sample = Sample(
        id_sample = None,
        fk_id_enddevice = 1, #temp_insert_dic['end_device_ids']['device_id'],
        temp = temp_insert_dic['uplink_message']['decoded_payload']['temp'],
        rh = temp_insert_dic['uplink_message']['decoded_payload']['rh'],
        pm1_0 = temp_insert_dic['uplink_message']['decoded_payload']['pm1_0'],
        pm2_5 = temp_insert_dic['uplink_message']['decoded_payload']['pm2_5'],
        pm10_0 = temp_insert_dic['uplink_message']['decoded_payload']['pm10_0']
    )
    
    samples.append(sample)


repo = SqlAlchemyRepository()
repo.init()

repo.create_registers(samples)

#wide_inserts_df['result'].to_list()
#wide_inserts_df['result'].astype(dic).to_dict('records')


OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/14/e3q8)